### 序列级奖励 vs. 词元级决策

- 在RLHF中，奖励模型 $r_{\psi}(x, y)$  对整个生成的序列 $y$  给出一个单一的标量分数。这个分数代表了人类对整个回应的偏好程度。
    - 语言模型的策略 $\pi_\phi(y_i | x, y_{<i})$ 是在每个词元级别上做出决策的。

- PPO 如何隐式完成信用分配
    - 价值函数 $V^\pi(s) = \mathbb{E}_{\pi} \left[ \sum_{k=0}^{\infty} \gamma^k R_{t+k+1} \Big| S_t = s \right]$
        - 由于主要的奖励 $r_\psi(x,y)$， 只在序列末尾给出，所以对于中间的词元 $y_t$，其即时奖励 $R_t$ 通常是 0；（或者只有KL散度项的贡献），最后token的奖励 $R_L = r_{\psi}(x,y)$
    - 优势函数 $A_t = Q(s_t, a_t) - V(s_t)$ => GAE
    - PPO（以及其他策略梯度方法）会根据优势函数来更新策略。如果一个词元选择 $y_t$ 使得优势函数 $A_t$ 为正（即这个选择比平均预期要好，因为它导向了一个更高最终奖励的序列），那么策略 $\pi_\phi$就会被调整，以增加在状态 $s_t$ 选择 $y_t$ 的概率；
        - 反之如果 $A_t$ 为负，则降低选择 $y_t$ 的概率
        - $\nabla_\phi J(\phi) \propto \mathbb{E}[\nabla_\phi \log \pi_\phi(y_t|s_t) \hat{A}_t]$
    - 以上两个函数都是 token 级别的；